In [1]:
import numpy as np
import pygame
from pygame import Color
from pygame.locals import KEYDOWN, KMOD_CTRL, QUIT, K_q
import cv2
import pyautogui
from cvzone.HandTrackingModule import HandDetector
import threading
import sys

# Constants and sound setup (no changes here)
fade_out = np.linspace(1, 0, 100)
fade_in = np.linspace(0, 1, 100)

bits = 16
sample_rate = 44100
max_sample = 2 ** (bits - 1) - 1

def gen_sound(buf, freq):
    n_samples = int(10 / 1000 * sample_rate)
    buf[:, 0] = (
        0.25
        * max_sample
        * np.sin(2 * np.pi * freq[0] * np.arange(n_samples) / sample_rate)
    )
    buf[:, 1] = (
        0.25
        * max_sample
        * np.sin(2 * np.pi * freq[1] * np.arange(n_samples) / sample_rate)
    )
    
    return pygame.sndarray.make_sound(buf)

def init_cap():
    cap = cv2.VideoCapture(0)
    cap.set(3, 1280)
    cap.set(4, 720)
    return cap

def init_detector():
    detector = HandDetector(detectionCon=0.8, maxHands=1)
    return detector

def move_cursor(cap, detector):
    while True:
        success, img = cap.read()
        if not success:
            continue
        hands, img = detector.findHands(img)
        
        if hands:
            pyautogui.moveTo(1380 - (7/6) * hands[0]["lmList"][8][0], (7/6)*hands[0]["lmList"][8][1]+200)

def main(argv: list = None):
    # Initialize sound
    pygame.mixer.pre_init(sample_rate, -bits, 2, 512)
    
    # Initialize pygame
    pygame.init()
    
    size = (800, 600)
    surf = pygame.display.set_mode(size)
    surf.fill(Color("black"))
    
    clock = pygame.time.Clock()
    
    FPS = 200
    dt = 1000 / FPS
    n_samples = int(10 / 1000 * sample_rate)
    mpos = None

    cap = init_cap()
    detector = init_detector()

    # Start move_cursor in a separate thread
    cursor_thread = threading.Thread(target=move_cursor, args=(cap, detector))
    cursor_thread.daemon = True  # Ensures the thread ends when the main program exits
    cursor_thread.start()

    # Create the initial buffer and sound
    buf = np.zeros((n_samples, 2), dtype=np.int16)
    sound = pygame.sndarray.make_sound(buf)
    buf = pygame.sndarray.samples(sound)  # Work with the same buffer reference
    gen_sound(buf, (440, 440))  # Generate initial sound
    sound.set_volume(0)
    sound.play(loops=-1, fade_ms=0)

    last_mpos = None  # To track the last mouse position

    # Event loop
    running = True
    while running:
        for e in pygame.event.get():
            if e.type == QUIT:  # Detecting when the user closes the window
                running = False
            if e.type == KEYDOWN:
                if e.mod & KMOD_CTRL and e.key == K_q:
                    running = False

        # Only update mouse position if the mouse is within the window
        if pygame.mouse.get_focused():
            mpos = pygame.mouse.get_pos()
        else:
            mpos = None

        # Only regenerate sound if the mouse has moved
        if mpos != last_mpos:
            last_mpos = mpos
            if mpos:
                freq = (mpos[0], mpos[1])
                sound.set_volume(0.5) # Use mouse position for frequency
            else:
                freq = (440, 440)
                sound.set_volume(0) # Default frequency when no mouse

            # Regenerate the sound buffer with the new frequency
            gen_sound(buf, freq)

        pygame.display.update()
        clock.tick(FPS)

    # Gracefully stop the sound and quit Pygame
    sound.stop()
    pygame.mixer.quit()
    pygame.quit()

    cap.release() 
    cv2.destroyAllWindows() 
    sys.exit(0)


if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


I0000 00:00:1736887231.209877  519556 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1736887231.219590  519901 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-01-14 15:40:31.224 python[8846:519556] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-14 15:40:31.224 python[8846:519556] +[IMKInputSession subclass]: chose IMKInputSession_Modern
W0000 00:00:1736887231.227673  519901 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736887232.030046  519900 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


SystemExit: 0

/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
